# TSA Chapter 1: Case Study - Romania Quarterly GDP

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch1/TSA_ch1_case_gdp/TSA_ch1_case_gdp.ipynb)

This notebook demonstrates:
- Romania quarterly GDP visualization (2010-2023)
- COVID-19 shock identification
- Time series decomposition into trend, seasonal, and residual components

In [ ]:
!pip install yfinance matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def add_legend_below(ax, ncol=3):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=False)

In [ ]:
# Download real Romania quarterly GDP from FRED
import warnings
warnings.filterwarnings('ignore')

FRED_URL = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=CLVMNACSCAB1GQRO'

try:
    gdp_df = pd.read_csv(FRED_URL, parse_dates=['DATE'])
    gdp_df = gdp_df.rename(columns={'DATE': 'date', 'CLVMNACSCAB1GQRO': 'gdp'})
    gdp_df['gdp'] = pd.to_numeric(gdp_df['gdp'], errors='coerce')
    gdp_df = gdp_df.dropna()
    gdp_df = gdp_df[(gdp_df['date'] >= '2010-01-01') & (gdp_df['date'] <= '2023-12-31')]
    quarters = gdp_df['date'].values
    gdp = gdp_df['gdp'].values / 1000  # Millions → Billions RON
    quarters = pd.to_datetime(quarters)
    print(f'Romania GDP loaded: {len(gdp)} observations from FRED')
except Exception as e:
    print(f'FRED download failed: {e}, using synthetic fallback')
    np.random.seed(42)
    quarters = pd.date_range('2010-01-01', '2023-12-31', freq='QS')
    n = len(quarters)
    trend = np.linspace(30, 80, n)
    seasonal = 3.0 * np.sin(2 * np.pi * np.arange(n) / 4 + 0.5)
    covid_shock = np.zeros(n)
    covid_idx = 40
    covid_shock[covid_idx] = -12
    covid_shock[covid_idx + 1] = -5
    covid_shock[covid_idx + 2] = 3
    noise = np.random.normal(0, 1.5, n)
    gdp = trend + seasonal + covid_shock + noise

print(f'{len(gdp)} quarterly observations')

In [ ]:
# Chart: ch1_case_gdp_raw
# Romania quarterly GDP raw data
fig, ax = plt.subplots(figsize=(7, 3.0))
ax.plot(quarters, gdp, color=COLORS['blue'], linewidth=1.0, marker='o', markersize=2, label='GDP Romania (bn RON)')
ax.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-09-01'), alpha=0.15, color=COLORS['red'], label='COVID-19')
ax.set_title('Quarterly GDP Romania (2010-2023)', fontsize=10, fontweight='bold')
ax.set_xlabel('Period')
ax.set_ylabel('GDP (bn RON, chained 2010)')
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y'))
ax.xaxis.set_major_locator(matplotlib.dates.YearLocator(2))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, ha='center')
add_legend_below(ax, ncol=2)
fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_case_gdp_raw')
plt.show()

In [ ]:
# Chart: ch1_case_gdp_decomposition
# GDP decomposition using STL
from statsmodels.tsa.seasonal import STL

gdp_series = pd.Series(gdp, index=pd.DatetimeIndex(quarters, freq='QS'))
stl = STL(gdp_series, period=4, robust=True)
result = stl.fit()

fig, axes = plt.subplots(4, 1, figsize=(7, 4.5), sharex=True)

axes[0].plot(quarters, gdp, color=COLORS['blue'], linewidth=0.8)
axes[0].set_ylabel('GDP')
axes[0].set_title('GDP Decomposition Romania (STL)', fontsize=10, fontweight='bold')

axes[1].plot(quarters, result.trend, color=COLORS['red'], linewidth=1.0)
axes[1].set_ylabel('Trend')

axes[2].plot(quarters, result.seasonal, color=COLORS['green'], linewidth=0.8)
axes[2].set_ylabel('Seasonal')

axes[3].plot(quarters, result.resid, color=COLORS['gray'], linewidth=0.5)
axes[3].axhline(0, color=COLORS['gray'], linewidth=0.3, linestyle='--')
axes[3].set_ylabel('Residual')
axes[3].set_xlabel('Period')

axes[3].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y'))
axes[3].xaxis.set_major_locator(matplotlib.dates.YearLocator(2))

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_case_gdp_decomposition')
plt.show()